# Lab 5: Open Loop Optimization

## Learning Objectives

1. Compute optimal heater strategy using physics-based model
2. Verify open loop performance with TC Lab hardware
3. Estimate unmeasured states (heater temperature) using model

## Installation

This notebook using Pyomo and Ipopt. Be sure to follow the installation instructions on the class website.


## Mathematical Model and Pyomo Simulation

\begin{align*}
C_p^H \frac{dT_H}{dt} & = U_a (T_{amb} - T_H) + U_b (T_S - T_H) + \alpha P u(t) + d(t)\\
C_p^S \frac{dT_S}{dt} & = - U_c (T_S - T_H) 
\end{align*}


### Pyomo Model

The code provides a library for performing several analysis tasks with our model. This is a `class`, which is a more sophisticated way to modularize our code. Briefly, classes store both data and methods to manipualte the data.

In [ ]:
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy as np

# Protip: only import the functions you need for each library.
# This tells other programs exactly where each function is defined.
# Avoid using from pyomo.environ import * as this imports everything
from pyomo.environ import ConcreteModel, Var, Param, Constraint, TransformationFactory, SolverFactory, Objective, minimize, value
from pyomo.dae import DerivativeVar, ContinuousSet

##### IMPORTANT #####
# Update the default values for Ua, Ub, CpH, and CpS to match your previous labs
#####################

Tamb = 21 # deg C

class TCLabPyomo:

    def __init__(self,
        mode, # specify mode
        t_data, # time data
        u_data, # input control data
        d_data, # disturbance data
        Tset_data, # set point data
        TS_data, # experimental data
        Tamb = Tamb, # ambient temperature deg C
        alpha = 0.00016, # watts / (units P1 * percent U1)
        P1 = 100, # max power, P1 units
        Ua = 0.0261, # heat transfer coefficient, watts/deg C
        Ub = 0.0222, # heat tranfer coefficient, watts/deg C
        CpH = 1.335, # heat transfer coefficient, joules/deg C
        CpS = 1.328 # heat transfer coefficient, joules/deg C        
    ):
        
        valid_modes = ['simulate','optimize','estimate','observe']

        if mode not in valid_modes:
            raise ValueError("'mode' must be one of the following:"+valid_modes)

        self.mode = mode
        self.t_data = t_data
        self.u_data = u_data
        self.d_data = d_data
        self.Tset_data = Tset_data
        self.TS_data = TS_data

        self.Tamb = Tamb
        self.Ua = Ua
        self.Ub = Ub
        self.CpH = CpH
        self.CpS = CpS
        self.alphaP = alpha*P1

        self._create_pyomo_model()

        return None

    def _create_pyomo_model(self):
            
        m = ConcreteModel()

        m.t = ContinuousSet(initialize = self.t_data)  # make sure the expt time grid are discretization points
        m.Th = Var(m.t, bounds=[0, 75])
        m.Ts = Var(m.t, bounds=[0, 75])

        def helper(my_array):
            assert len(my_array) == len(self.t_data), "Dimension mismatch."
            data = {}
            for k,t in enumerate(self.t_data):
                data[t] = my_array[k]
            return data

        if self.mode in ['simulate', 'observe', 'estimate']:
            m.U = Param(m.t, initialize=helper(self.u_data))
        else:
            m.U = Var(m.t, bounds=(0, 100))

        if self.mode in ['simulate', 'optimize', 'estimate']:

            if self.d_data is None:
                 m.D = Param(m.t, default = 0)
            else:
                m.D = Param(m.t, initialize=helper(self.d_data))

        else:
            m.D = Var(m.t)

        m.Tamb = Param(initialize=self.Tamb)
        m.alphaP = Param(initialize=self.alphaP)

        if self.mode in ['simulate', 'optimize', 'observe']:
            m.Ua = Param(initialize=self.Ua)
            m.Ub = Param(initialize=self.Ub)
            m.CpH = Param(initialize=self.CpH)
            m.CpS = Param(initialize=self.CpS)
        else: # estimate
            m.Ua = Var(initialize=self.Ua, bounds=(1E-5, 1.0))
            m.Ub = Var(initialize=self.Ub, bounds=(1E-5, 1.0))
            m.CpH = Var(initialize=self.CpH, bounds=(1E-3,10))
            m.CpS = Var(initialize=self.CpS, bounds=(1E-3, 10))

        if self.mode == 'optimize':
            m.Tset = Param(m.t, initialize=helper(self.Tset_data))

        if self.mode in ['estimate', 'observe']:
            m.Ts_measure = Param(m.t, initialize=helper(self.TS_data))

        m.Thdot = DerivativeVar(m.Th, wrt = m.t)
        m.Tsdot = DerivativeVar(m.Ts, wrt = m.t)

        # differential equations
        m.Th_ode = Constraint(m.t, rule = lambda m, t: 
                            m.CpH*m.Thdot[t] == m.Ua*(m.Tamb - m.Th[t]) + m.Ub*(m.Ts[t] - m.Th[t]) + m.alphaP*m.U[t] + m.D[t])
        
        m.Ts_ode = Constraint(m.t, rule = lambda m, t: 
                            m.CpS*m.Tsdot[t] == m.Ub*(m.Th[t] - m.Ts[t]))

        if self.mode == 'optimize':
            
            # Add requested constraints on U ramping here
            m.Udot = DerivativeVar(m.U, wrt = m.t)

            # Add your solution here

        TransformationFactory('dae.finite_difference').apply_to(m, scheme='BACKWARD',nfe=len(self.t_data)-1)

        if self.mode == 'optimize':
            # defining the tracking objective function
            m.obj = Objective(expr=sum( (m.Ts[t] - m.Tset[t])**2 + 0.01*(m.Th[t] - m.Tset[t])**2 for t in m.t), sense=minimize)

        if self.mode == 'observe':
            # define observation (state estimation)
            m.obj = Objective(expr=sum((m.Ts[t] - m.Ts_measure[t])**2 + 0.01*m.D[t]**2 for t in m.t), sense=minimize)

        if self.mode == 'estimate':
            # define parameter estimation objective
            m.obj = Objective(expr=sum((m.Ts[t] - m.Ts_measure[t])**2 for t in m.t), sense=minimize)

        # initial conditions
        m.Th[0].fix(m.Tamb)
        m.Ts[0].fix(m.Tamb)

        # input specifications
        '''
        if mode == 'simulate':
            for k,t in enumerate(t_data):
                m.U[t].fix(u_data[k])
        if mode in ['simulate','optimize']:
            for k,t in enumerate(t_data):
                m.D[t].fix(d_data[k])

        '''

        self.m = m
    
    def solve(self):

        SolverFactory('ipopt').solve(self.m, tee=True)

    def get_time(self):
        return self.t_data

    def get_Th(self):
        return np.array([value(self.m.Th[t]) for t in self.t_data])
    
    def get_Ts(self):
        return np.array([value(self.m.Ts[t]) for t in self.t_data])
    
    def get_U(self):
        return np.array([value(self.m.U[t]) for t in self.t_data])
    
    def get_D(self):
        return np.array([value(self.m.D[t]) for t in self.t_data])
    
    def get_parameters(self):
        return value(self.Ua), value(self.Ub), value(self.CpH), value(self.CpS)

    def print_parameters(self):

        Ua, Ub, CpH, CpS = self.get_parameters()

        print("The value of Ua is", round(Ua,4), "Watts/degC.")
        print("The value of Ub is", round(Ub,4), "Watts/degC.")
        print("The value of CpH is", round(CpH,3), "Joules/degC.")
        print("The value of CpS is", round(CpS,3),"Joules/degC.")

    def plot(self):

        # Extract predictions
        Th = self.get_Th()
        Ts = self.get_Ts()
        U = self.get_U()
        D = self.get_D()

        # create figure
        plt.figure(figsize=(10,6))

        # subplot 1: temperatures
        plt.subplot(3, 1, 1)
        if self.TS_data is not None:
            plt.scatter(self.t_data, self.TS_data, marker='.', label="$T_{S}$ measured", alpha=0.5,color='green')
        plt.plot(self.t_data, Th, label='$T_{H}$ predicted')
        plt.plot(self.t_data, Ts, label='$T_{S}$ predicted')
        if self.Tset_data is not None:
            plt.plot(self.t_data, self.Tset_data, label='$T_{set}$')

        plt.title('temperatures')
        plt.ylabel('deg C')
        plt.legend()
        plt.grid(True)

        # subplot 2: control decision
        plt.subplot(3, 1, 2)
        plt.plot(self.t_data, U)
        plt.title('heater power')
        plt.ylabel('percent of max')
        plt.grid(True)

        # subplot 3: disturbance
        plt.subplot(3, 1, 3)
        plt.plot(self.t_data, D)
        plt.title('disturbance')
        plt.ylabel('watts')
        plt.grid(True)

        plt.tight_layout()
        plt.show()

Take a few minutes to study this code. A few features:
* It supports four modes:
  * `simulate` solve the model with all of the inputs specified (zero degrees of freedom)
  * `optimize` determine the best $u(t)$ to track the desired $T_{set}(t)$
  * `observe` estimate the unmeasured state $T_{H}(t)$ and disturbance $d(t)$ from experimental data
  * `estimate` estimate the model parameters $U_a$, $U_b$, $C_p^H$, $C_p^S$ from experimental data
* The `__init__` method is called automatically when you create an instance of the object. The initial method is how you pass in data
* The `solve` method call the numerical optimization algorithm
* The `plot` method plots the Pyomo results
* The `get_Ts`, `get_Th`, `get_U`, and `get_D` methods return data stored in the Pyomo model after it is solved

This notebook walks you through using the library. The take away message is that a class is a convient way to modularize our code and maximize reuse. The four supported modes are very similar.

### Process Inputs

The next cell defines some process inputs that will be used throughout the notebook to demonstrate aspects of process simulation, control, and estimation.  These are gathered in one place to make it easier to modify the notebook to test the response under different conditions. These functions are implemented using the `interp1d` from the `scipy` library.

In [ ]:
%matplotlib inline

tclab_disturbance = interpolate.interp1d(
    [ 0, 300, 400, 9999],   # time
    [ 0, 0, -.5, -.5])      # disturbance value

tclab_input = interpolate.interp1d(
    [ 0, 50, 51, 450, 451, 9999],   # time
    [ 0,  0, 80,  80,   25,   25])  # input value

tclab_setpoint = interpolate.interp1d(
    [0, 50, 150, 450, 550, 9999],   # time 
    [Tamb, Tamb, 60, 60, 35, 35])   # set point value

t_sim = np.linspace(0, 1000, 201)       # create 201 time points between 0 and 1000 seconds
u_sim = tclab_input(t_sim)              # calculate input signal at time points
d_sim = tclab_disturbance(t_sim)        # calculate disturbance at time points
setpoint_sim = tclab_setpoint(t_sim)    # calculate set point at time points

plt.figure(figsize=(10,6))
plt.subplot(3, 1, 1)
plt.plot(t_sim, setpoint_sim)
plt.title('setpoint')
plt.ylabel('deg C')
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(t_sim, u_sim)
plt.title('heat power input')
plt.ylabel('percent of max')
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(t_sim, d_sim)
plt.title('unmeasured disturbance')
plt.ylabel('watts')
plt.grid(True)

plt.tight_layout()

Let's see how well our initial guess at a control strategy will work for us.

\begin{align*}
C_p^H \frac{dT_H}{dt} & = U_a (T_{amb} - T_H) + U_b (T_S - T_H) + \alpha P u(t) + d(t)\\
C_p^S \frac{dT_S}{dt} & = - U_b (T_S - T_H) 
\end{align*}

subject to initial conditions

\begin{align*}
T_H(t_0) & = T_{amb} \\
T_S(t_0) & = T_{amb}
\end{align*}

and prior specification of inputs $u(t)$ and $d(t)$.

In [ ]:
sim = TCLabPyomo('simulate',
                t_sim,
                u_sim,
                d_sim,
                setpoint_sim,
                None
)

sim.solve()
sim.plot()

## Optimal Control with Knowledge of Disturbances

An optimal control policy minimizes the differences

\begin{align*}
\min_{u} \int_{t_0}^{t_f} \|T_S^{SP}(t) - T_S(t)\|^2\,dt \\
\end{align*}

subject to constraints

\begin{align*}
C_p^H \frac{dT_H}{dt} & = U_a (T_{amb} - T_H) + U_c (T_S - T_H) + P u(t) + d(t)\\
C_p^S \frac{dT_S}{dt} & = - U_c (T_S - T_H) 
\end{align*}

initial conditions

\begin{align*}
T_H(t_0) & = T_{amb} \\
T_S(t_0) & = T_{amb}
\end{align*}

and prior knowledge of $d(t)$.

## Exercise 1

The optimal control computed above requires rapid changes in power level. In process systems where control action requires movement of a valve stem position, there are often limits on how fast the manipulated variable can change. Modify the model to include differential inequalities that limit the time rate of change of control.

\begin{align*}
\frac{du}{dt} & \leq \dot{u}_{max} \\
\frac{du}{dt} & \geq -\dot{u}_{max}
\end{align*}

where $\dot{u}_{max}$ is the maximum rate of change. 

In [ ]:
# Be sure to check and update the ambient temperature!
# Hint: you can do this with the hardware

opt = TCLabPyomo('optimize',
                 t_sim,
                 u_sim,
                 d_sim,
                 setpoint_sim,
                 None
)

opt.solve()
opt.plot()

## Exercise 2

Implement your calculated solution on the TC Lab hardware. Save the simulation results to a text file.

In [ ]:
t_solution = opt.get_time()
u_solution = opt.get_U()
TS_solution = opt.get_Ts()
TH_solution = opt.get_Th()

u_open_loop = interpolate.interp1d(
    t_solution,   # time
    u_solution)   # control value

TS_predict = interpolate.interp1d(
    t_solution,   # time
    TS_solution)   # sensor temperature prediction

TH_predict = interpolate.interp1d(
    t_solution,   # time
    TH_solution)   # sensor temperature prediction

from tclab import TCLab, clock, Historian, Plotter, setup

TCLab = setup(connected=True)

t_final = t_solution[-1] # change this to 500 seconds for the actual experiment
t_step = 0.1
with TCLab() as lab:
    sources = [("T1", lambda: lab.T1), ("T2", lambda: lab.T2),
               ("T1pred", lambda: TS_predict(t)),
               ("SP1", lambda: tclab_setpoint(t)),
               ("Q1", lab.Q1), ("Q2", lab.Q2)]
    h = Historian(sources)
    p = Plotter(h, t_final, layout=(("T1", "SP1","T1pred"), ("T2",), ("Q1", "Q2")))
    for t in clock(t_final, t_step):
        U1 = u_open_loop(t)
        lab.Q1(U1)
        p.update()

In [ ]:
import os.path

def save_tclab_data(h, file_name, overwrite_file=False):
    ''' Save TCLab data to csv file

    Arguments:
        h: tclab historian objective
        file_name: valid file names as a string
        overwrite_file: bool, if True, overwrite exisiting file
            default is False to safeguard against accidentally rerunning this function


    '''

    if not overwrite_file and os.path.isfile('./'+file_name):
        raise FileExistsError(file_name + ' already exisits. Either choose a new filename or set overwrite_file = True.')
    else:
        h.to_csv(file_name)
        print("Successfully saved data to "+file_name)

    return file_name



data_file = save_tclab_data(h,
                'tclab-open-loop1.csv', # tip: change this to end in "2" when copying code later in assignment
                False
)




## Estimation/Observation

> "... and now my watch begins ..." ―The Night's Watch oath, Game of Thrones

The trouble with open-loop optimal control is that we cannot anticipate or know the values of unmeasured disturbances, much less the future values of those disturbances. The best we can do is use available data and process models to estimate the process state and disturbances. The state estimation problem is:


\begin{align*}
\min_{\hat{T}_H, \hat{T}_S, \hat{d}} \int_{t - h}^t \|\hat{T}_S(t) - T_S(t)\|^2\,dt \\
\end{align*}

subject to

\begin{align*}
C_p^H \frac{d\hat{T}_H}{dt} & = U_a (T_{amb} - \hat{T}_H) + U_c (\hat{T}_S - \hat{T}_H) + P u(t) + \hat{d}(t)\\
C_p^S \frac{d\hat{T}_S}{dt} & = - U_c (\hat{T}_S - \hat{T}_H) 
\end{align*}

and initial conditions

\begin{align*}
T_H(t_0) & = T_{amb} \\
T_S(t_0) & = T_{amb}
\end{align*}


## Exercise 3

Estimate the heater temperature and disturbance from your simulation data.

In [ ]:
import pandas as pd
experiment_data = pd.read_csv(data_file)
experiment_data.head()


In [ ]:
time_data = experiment_data['Time'].array # grab column from pandas, convert to array
time_data -= time_data[0] # make initial time 0

u_data = experiment_data['Q1'].array
T1_data = experiment_data['T1'].array

obs = TCLabPyomo('observe',
                 time_data,
                 u_data,
                 None,
                 None,
                 T1_data)

obs.solve()
obs.plot()

## Parameter Estimation

It is also possible our model does not match the physics of the system (e.g., wrong assumptions) or our estimated parameters are no longer valid or both. To investigate the later, we can repeat the parameter estimation problem for earlier in the semester:


\begin{align*}
\min_{\hat{T}_H, \hat{T}_S, \hat{U_a}, \hat{U_b}, \hat{C_p^H}, \hat{C_p^S}} \int_{t - h}^t \|\hat{T}_S(t) - T_S(t)\|^2\,dt \\
\end{align*}

subject to

\begin{align*}
\hat{C_p^H} \frac{d\hat{T}_H}{dt} & = \hat{U_a} (T_{amb} - \hat{T}_H) + \hat{U_b} (\hat{T}_S - \hat{T}_H) + \alpha P u(t) + d(t)\\
\hat{C_p^S} \frac{d\hat{T}_S}{dt} & = - \hat{U_b} (\hat{T}_S - \hat{T}_H) 
\end{align*}

and initial conditions

\begin{align*}
T_H(t_0) & = T_{amb} \\
T_S(t_0) & = T_{amb}
\end{align*}

and assumption disturbance is zero

\begin{equation}
d(t) = 0
\end{equation}


## Exercise 4

Using data from your experiment, restimate the model parameters $U_a$, $U_b$, $C_p^H$, $C_p^S$.

In [ ]:
est = TCLabPyomo('estimate',
                 time_data,
                 u_data,
                 None,
                 None,
                 T1_data)

est.solve()
est.plot()

In [ ]:
Ua, Ub, CpH, CpS = est.get_parameters()

est.print_parameters()

## Exercise 5

Compare the profiles of $T_H$, $T_S$, $Q_1$, and $d$ (disturbance) across:
* The open loop optimization (prediction)
* The hardware results
* The state estimation

Write a short paragraph or a bulleted list of observations.

**Your Observations:**

Compare the mean absolute and mean squared tracking error $T_S - T_{set}$ and $T_H - T_{set}$ across:
* The open loop optimization
* The state estimation (using hardware results)

Write a short paragraph or a bulleted list of observations.

## Excerise 6

Repeat the procedure from this lab for the chocolate tempering profile.

### Open Loop Optimization

In [ ]:
# Be sure to check and update the ambient temperature!

# Add your solution here

### Hardware Implementation

In [ ]:
# Add your solution here

In [ ]:
# Add your solution here

### State Estimation

In [ ]:
# Add your solution here

## Parameter Estimation

In [ ]:
# Add your solution here

In [ ]:
est2.print_parameters()

### Discussion

## Coding the Observer as a Python Generator

In [ ]:
from pyomo.environ import *
from pyomo.dae import *

def tclab_observer(h=2):
    t_hist = [-1]
    u_hist = [0]
    Ts_hist = [Tamb]
    
    t_est = -1
    Th_est = []
    Ts_est = []
    d_est = []
    
    while True:
        t_meas, u_meas, Ts_meas = yield t_est, Th_est, Ts_est, d_est
  
        t_hist.append(t_meas)
        u_hist.append(u_meas)
        Ts_hist.append(Ts_meas)
        
        t_hist = t_hist[-h:]
        u_hist = u_hist[-h:]
        Ts_hist = Ts_hist[-h:]
        
        m = ConcreteModel()
        m.t = ContinuousSet(initialize = t_hist)
        m.Th = Var(m.t)
        m.Ts = Var(m.t)
        m.U = Var(m.t, bounds=(0, 100))
        m.D = Var(m.t)

        m.Thdot = DerivativeVar(m.Th, wrt = m.t)
        m.Tsdot = DerivativeVar(m.Ts, wrt = m.t)

        m.Th_ode = Constraint(m.t, rule = lambda m, t: 
                          CpH*m.Thdot[t] == Ua*(Tamb - m.Th[t]) + Uc*(m.Ts[t] - m.Th[t]) + P*m.U[t] + m.D[t])
        m.Ts_ode = Constraint(m.t, rule = lambda m, t: 
                          CpS*m.Tsdot[t] == Uc*(m.Th[t] - m.Ts[t]))

        m.Usim = Constraint(range(0, len(t_hist)), rule = lambda m, k: m.U[t_hist[k]] == u_hist[k])

        m.ls_observer = sum([(m.Ts[t_hist[k]] - Ts_hist[k])**2 for k in range(0, len(t_hist))])
        m.obj = Objective(expr = m.ls_observer, sense=minimize)

        TransformationFactory('dae.finite_difference').apply_to(m, nfe=len(t_hist), method='forward')
        SolverFactory('ipopt', executable=ipopt_executable).solve(m)
    
        t_est = t_hist[-1]
        Th_est = m.Th[t_est]()
        Ts_est = m.Ts[t_est]()
        d_est = m.D[t_est]()

In [ ]:
%%time

t_est = []
Th_est = []
Ts_est = []
d_est = []

observer = tclab_observer(5)
observer.send(None)

for k in range(0, len(t_sim)):
    t, Th, Ts, d = observer.send([t_sim[k], u_sim[k], Ts_sim[k]])
    t_est.append(t)
    Th_est.append(Th)
    Ts_est.append(Ts)
    d_est.append(d)
    
plt.figure(figsize=(10,8))
plt.subplot(3,1,1)
plt.plot(t_est, Th_est)
plt.plot(t_sim, Th_sim)
plt.grid(True)

plt.subplot(3,1,2)
plt.plot(t_est, Ts_est)
plt.plot(t_sim, Ts_sim)
plt.grid(True)

plt.subplot(3,1,3)
plt.plot(t_est, d_est)
plt.plot(t_sim, d_sim)
plt.grid(True)

plt.tight_layout()

## Coding the Controller as a Python Generator

In [ ]:
from pyomo.environ import *
from pyomo.dae import *

def tclab_control(h=100):
    u = 0
    while True:
        t_est, Th_est, Ts_est, d_est = yield u
        
        tf = t_est + h
        m = ConcreteModel()
        m.t = ContinuousSet(initialize=np.linspace(t, tf, 1 + round((tf-t)/2)))
        m.Th = Var(m.t)
        m.Ts = Var(m.t)
        m.U = Var(m.t, bounds=(0, 100))
        m.Thdot = DerivativeVar(m.Th, wrt = m.t)
        m.Tsdot = DerivativeVar(m.Ts, wrt = m.t)
        m.heater = Constraint(m.t, rule = lambda m, t: 
                          CpH*m.Thdot[t] == Ua*(Tamb - m.Th[t]) + Uc*(m.Ts[t] - m.Th[t]) + P*m.U[t] + d)
        m.sensor = Constraint(m.t, rule = lambda m, t: 
                          CpS*m.Tsdot[t] == Uc*(m.Th[t] - m.Ts[t]))
        m.Th[t].fix(Th_est)
        m.Ts[t].fix(Ts_est)
        m.obj = Objective(expr = sum([(tclab_setpoint(t) - m.Th[t])**2 for t in m.t]), sense=minimize)
        TransformationFactory('dae.finite_difference').apply_to(m, nfe=len(m.t), method='backwards')
        SolverFactory('ipopt', executable=ipopt_executable).solve(m)
        
        umpc = np.array([m.U[t]() for t in m.t])
        plt.plot([t for t in m.t], umpc)
        u = umpc[0]

observer = tclab_observer(3)
observer.send(None)

controller = tclab_control()
controller.send(None)


t_mpc = []
u_mpc = []
Th_mpc = []
Ts_mpc = []


for k in range(0, len(t_sim)):
    t, Th, Ts, d = observer.send([t_sim[k], u_sim[k], Ts_sim[k]])
    u = controller.send([t, Th, Ts, d])
    u_mpc.append(u)
    Th_mpc.append(Th)
    Ts_mpc.append(Ts)
    
plt.plot(t_sim, u_mpc)
plt.plot(t_sim, Th_mpc)

## MPC Demonstration

In [ ]:
from tclab import setup, clock, Historian, Plotter
TCLab = setup(connected=False, speedup=20)

tf = 800        # run time

# create a controller instance
controller = tclab_control(600)
controller.send(None)

# create an model estimator
observer = tclab_observer(5)
observer.send(None)

# execute the event loop
tf = 800
with TCLab() as lab:
    h = Historian([('T1', lambda: lab.T1), ('Q1', lab.Q1),
                   ('Th', lambda: Th), ('Ts', lambda: Ts)])
    p = Plotter(h, tf)
    U1 = 0
    for t in clock(tf, 5):                    # allow time for more calculations
        T1 = lab.T1                           # measure the sensor temperature
        t, Th, Ts, d = observer.send([t, U1, T1])  # estimate the heater temperature                      # get setpoint
        U1 = controller.send([t, Th, Ts, d])        # compute control action
        lab.U1 = U1                           # set manipulated variable  
        p.update(t)                           # log data

## To be Removed

In [ ]:
def tclab_control(t, h, ic, d):
    tf = t + h
    
    m = ConcreteModel()
    m.t = ContinuousSet(initialize=np.linspace(t, tf, 1 + round((tf-t)/2)))
    m.Th = Var(m.t)
    m.Ts = Var(m.t)
    m.U = Var(m.t, bounds=(0, 100))
    m.Thdot = DerivativeVar(m.Th, wrt = m.t)
    m.Tsdot = DerivativeVar(m.Ts, wrt = m.t)
    m.heater = Constraint(m.t, rule = lambda m, t: 
                      CpH*m.Thdot[t] == Ua*(Tamb - m.Th[t]) + Uc*(m.Ts[t] - m.Th[t]) + P*m.U[t] + d)
    m.sensor = Constraint(m.t, rule = lambda m, t: 
                      CpS*m.Tsdot[t] == Uc*(m.Th[t] - m.Ts[t]))
    m.Th[t].fix(ic[0])
    m.Ts[t].fix(ic[1])
    m.obj = Objective(expr = sum([(tclab_setpoint(t) - m.Th[t])**2 for t in m.t]), sense=minimize)
    TransformationFactory('dae.finite_difference').apply_to(m, nfe=len(m.t), method='backwards')
    SolverFactory('ipopt', executable=ipopt_executable).solve(m)
    tsim = np.array([t for t in m.t])
    Thsim = np.array([m.Th[t]() for t in m.t])
    Tssim = np.array([m.Ts[t]() for t in m.t])
    Usim = np.array([m.U[t]() for t in m.t])
    return [tsim, Usim, Thsim, Tssim]
    
[tsim, Usim, Thsim, Tssim] = tclab_control(100, 800, [30, 30], .5)
    
# visualization

plt.subplot(2,1,1)
plt.plot(tsim, Thsim, tsim, Tssim)
plt.title('temperatures')
plt.ylabel('deg C')
plt.legend(['T_heater', 'T_sensor'])
plt.grid(True)

plt.subplot(2,1,2)
plt.plot(tsim, Usim)
plt.title('heater input')
plt.ylabel('percent of max')
plt.grid(True)

plt.tight_layout()